# Wine Quality — Extra Trees Regression (one command per code cell)
# Extremely randomized trees ensemble; faster and robust on tabular data.


# Imports

In [ ]:
import pandas as pd  # data handling


In [ ]:
import numpy as np  # numeric helpers


In [ ]:
from sklearn.model_selection import train_test_split, cross_validate  # split and CV


In [ ]:
from sklearn.ensemble import ExtraTreesRegressor  # extra trees regressor


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score  # metrics


# Load dataset

In [ ]:
DATASET = 'red'  # 'red' or 'white'


In [ ]:
CSV_PATH = f"/Users/sb/Documents/Study-Material/Machine-Learning/winequality-{DATASET}.csv"  # dataset path


In [ ]:
df = pd.read_csv(CSV_PATH, sep=';')  # load CSV


# Split

In [ ]:
X = df.drop(columns=['quality'])  # features


In [ ]:
y = df['quality']  # target


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # split


# Model

In [ ]:
xtr = ExtraTreesRegressor(n_estimators=300, random_state=42, n_jobs=None)  # extremely randomized trees ensemble


In [ ]:
xtr.fit(X_train, y_train)  # fit model


In [ ]:
pred = xtr.predict(X_test)  # predictions


# Evaluate

In [ ]:
mae = mean_absolute_error(y_test, pred)  # MAE


In [ ]:
rmse = mean_squared_error(y_test, pred, squared=False)  # RMSE


In [ ]:
r2 = r2_score(y_test, pred)  # R^2


In [ ]:
{'MAE': mae, 'RMSE': rmse, 'R2': r2}  # metrics


# Cross-validation

In [ ]:
cv = cross_validate(xtr, X, y, cv=5, scoring=('r2','neg_root_mean_squared_error'))  # 5-fold CV


In [ ]:
{k: (np.mean(v), np.std(v)) for k, v in cv.items() if k.startswith('test_')}  # CV summary


# Feature importance (optional)

In [ ]:
dict(sorted({col: imp for col, imp in zip(X.columns, xtr.feature_importances_)}.items(), key=lambda kv: kv[1], reverse=True))  # importance by feature


# MLflow Tracking — log params, metrics, and model

In [ ]:
import mlflow  # experiment tracking


In [ ]:
import mlflow.sklearn  # sklearn flavor logging


In [ ]:
mlflow.set_experiment('ExtraTreesRegression-WineQuality')  # experiment per algorithm


In [ ]:
mlflow.start_run(run_name=f"{DATASET}")  # start run


In [ ]:
mlflow.log_params({'n_estimators': getattr(xtr, 'n_estimators', None), 'max_depth': getattr(xtr, 'max_depth', None), 'random_state': getattr(xtr, 'random_state', None)})  # params


In [ ]:
mlflow.log_metric('MAE', mae)  # MAE


In [ ]:
mlflow.log_metric('RMSE', rmse)  # RMSE


In [ ]:
mlflow.log_metric('R2', r2)  # R2


In [ ]:
mlflow.sklearn.log_model(xtr, artifact_path='model')  # log model


In [ ]:
mlflow.end_run()  # end run


# Persistence — save to disk and reload

In [ ]:
import os  # fs utils


In [ ]:
import joblib  # persistence


In [ ]:
from datetime import datetime  # timestamp


In [ ]:
os.makedirs('Machine-Learning/models/ExtraTreesRegression', exist_ok=True)  # ensure dir


In [ ]:
MODEL_PATH = f"Machine-Learning/models/ExtraTreesRegression/extra_trees_wine_{DATASET}_" + datetime.now().strftime('%Y%m%d_%H%M%S') + ".joblib"  # path


In [ ]:
joblib.dump(xtr, MODEL_PATH)  # save model


In [ ]:
loaded = joblib.load(MODEL_PATH)  # load model


In [ ]:
float(xtr.predict(X_test.iloc[[0]])[0])  # single-row prediction
